In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import pickle

In [9]:
url = 'https://raw.githubusercontent.com/datasets/finance-vix/main/data/vix-daily.csv'  # Example dataset URL
data = pd.read_csv(url)
data['DATE'] = pd.to_datetime(data['DATE'])
data.set_index('DATE', inplace=True)

In [10]:
data.head()

,OPEN,HIGH,LOW,CLOSE
DATE,,,,
1990-01-02,17.24,17.24,17.24,17.24
1990-01-03,18.19,18.19,18.19,18.19
1990-01-04,19.22,19.22,19.22,19.22
1990-01-05,20.11,20.11,20.11,20.11
1990-01-08,20.26,20.26,20.26,20.26


In [11]:
close_prices = data['CLOSE'].values.reshape(-1, 1)

In [12]:
# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close_prices = scaler.fit_transform(close_prices)


In [13]:
# Step 4: Create Features and Targets
# Define the sequence length (e.g., 60 days)
sequence_length = 60


In [14]:
X, y = [], []
for i in range(sequence_length, len(scaled_close_prices)):
    X.append(scaled_close_prices[i-sequence_length:i, 0])
    y.append(scaled_close_prices[i, 0])


In [15]:
X, y = np.array(X), np.array(y)

# Reshape X to be compatible with LSTM (samples, time_steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [16]:
# Step 5: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:

# Step 6: Build the LSTM Model
model = Sequential()



In [18]:
# Add LSTM layers
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))



In [20]:
# Add Dense layers
model.add(Dense(units=25))
model.add(Dense(units=1))



In [21]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')



In [ ]:
# Step 7: Train the Model
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))



Epoch 1/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step - loss: 0.0046 - val_loss: 0.0011
Epoch 2/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 20s 67ms/step - loss: 0.0012 - val_loss: 9.2177e-04
Epoch 3/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - loss: 0.0011 - val_loss: 9.0195e-04
Epoch 4/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step - loss: 0.0010 - val_loss: 6.9553e-04
Epoch 5/50
112/220 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - loss: 7.7081e-04

In [ ]:
# Step 8: Evaluate the Model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')


In [ ]:

# Plot the training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()
